In [4]:

from sklearn.feature_extraction.text import CountVectorizer
document1 = 'Dog hates a cat It loves to go out and play'
document2 = 'Cat loves to play with a ball'

# converting sentences to lower case
document1 = document1.lower()
document2 = document2.lower()

# Intialize BoWs
count_vect = CountVectorizer()

# fit the corpus to CountVectorizer
count_vect.fit([document1, document2])

#-print("feature names ", count_vect.get_feature_names())

# bag of word representation of document1
bow1 = count_vect.transform([document1])
t1=bow1.toarray()
print(t1)

# bag of word representation of document2
bow2 = count_vect.transform([document2])
t2=bow2.toarray()
print( t2)

[[1 0 1 1 1 1 1 1 1 1 1 0]]
[[0 1 1 0 0 0 0 1 0 1 1 1]]


#TRAINING MODEL

In [5]:

from matplotlib import pyplot
import torch
import torch.nn as nn
X_test = torch.tensor(t1, dtype=torch.float32)
Y = torch.tensor(t2, dtype=torch.float32)
n_samples, n_features = X_test.shape

# 0) create a test sample
X_test = torch.tensor([5], dtype=torch.float32)
model = nn.Linear(n_samples,n_features)

# 2) Define loss and optimizer
learning_rate = 0.01
n_iters = 100
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop
for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X_test)

    # loss
    l = loss(Y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

   # zero the gradients after updating
    optimizer.zero_grad()
    
#     if epoch % 10 == 0:
print('loss = ', l )    
print()
print("model parameterrs for best accuracy ------------------------")
l= model.named_parameters()
print(l)
# for i,j in model.named_parameters():
#             print(i,j )
# print()

print(model.weight,model.bias)
print("prediction before trainig :",Y)
print("hyper parameters- for best accuracy --------------------------------")
print("learning rate",learning_rate,    "number of epochs:",epoch)
# print(f"Prediction after training: {model(X_test).item():.3f}")


loss =  tensor(0.0017, grad_fn=<MseLossBackward0>)

model parameterrs for best accuracy ------------------------
<generator object Module.named_parameters at 0x7fc3948a2ff0>
Parameter containing:
tensor([[ 0.0458],
        [ 0.1724],
        [ 0.1432],
        [-0.0563],
        [-0.0877],
        [-0.1679],
        [-0.0926],
        [ 0.2831],
        [-0.1511],
        [ 0.3255],
        [ 0.1383],
        [ 0.3398]], requires_grad=True) Parameter containing:
tensor([-0.1962,  0.0777,  0.2654,  0.3127,  0.5040,  0.8013,  0.4667, -0.4754,
         0.7866, -0.6253,  0.2591, -0.6972], requires_grad=True)
prediction before trainig : tensor([[0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.]])
hyper parameters- for best accuracy --------------------------------
learning rate 0.01 number of epochs: 99


# after keeping learning rate to half and observing prediction .



In [6]:
import torch.nn as nn
learning_rate = 0.01/2
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

#3) Training loop
for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X_test)

    # loss
    l = loss(Y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()
print(' loss = ', l)
print()
print("model parameterrs for best accuracy ------------------------")

print()
print("hyper parameters- for best accuracy --------------------------------")
print("learning rate",learning_rate,    "number of epochs:",epoch)
print(f"Prediction after training: {model(X_test)}")

    
    
    

 loss =  tensor(2.0505e-05, grad_fn=<MseLossBackward0>)

model parameterrs for best accuracy ------------------------

hyper parameters- for best accuracy --------------------------------
learning rate 0.005 number of epochs: 99
Prediction after training: tensor([ 3.6680e-03,  9.9327e-01,  9.9794e-01,  3.5175e-03,  7.3407e-03,
        -4.2624e-03,  4.0108e-04,  9.9329e-01,  3.4754e-03,  1.0003e+00,
         9.9445e-01,  1.0002e+00], grad_fn=<AddBackward0>)


# observation :accuracy improved for small learning rate 

In [7]:
import numpy as np 
import torch
import pandas as pd 
data_hound = pd.read_csv("hound.txt", delimiter='\t', header=None)
data_hound.columns = ["Review_text", "Review_class"]

data_sample = pd.read_csv("sample.txt", delimiter='\t', header=None)
data_sample.columns = ["Review_text", "Review_class"]

# data_yelp = pd.read_csv("yelp_labelled.txt", delimiter='\t', header=None)
# data_yelp.columns = ["Review_text", "Review_class"]

data = pd.concat([data_hound,data_sample])
data_hound.loc[1,["Review_text"]]
print(data[:2])
import re
import nltk

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

def clean_text(df):
    all_reviews = list()
    lines = df["Review_text"].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
#         words = [w for w in words if not w in stop_words]
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

all_reviews = clean_text(data)
#all_reviews[0:20]
stopwords.words("english")

from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(min_df=3)   
X = CV.fit_transform(all_reviews).toarray()
y = data.loc[:,["Review_class"]]
print(type(X))
print(type(y))
y=y.dropna()
print(X[1])
print(y)

X=torch.tensor(X,dtype=torch.float32)
y=y.to_numpy()#we cant chge directly from data frame  to tensor so 1st np array  then  torch
y=torch.tensor(y,dtype=torch.float32)
print(X.shape)


                                         Review_text  Review_class
0  So there is no way for me to plug it in here i...             0
1                        Good case, Excellent value.             1


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/local/ZOHOCORP/abdul-pt6532/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [5]:
from matplotlib import pyplot
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
# X = torch.tensor(X, dtype=torch.float32)
# Y = torch.tensor(y, dtype=torch.float32)------------1




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(X_train)
n_samples, n_features = X_train.shape
print(X_train.shape)
model = nn.Linear(n_features,1)

learning_rate = 0.001
n_iters = 100
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# print("i",X_train[1])
y_predicted = model(X_train[1])
print("befr t",y_predicted)
# 3) Training loop
for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X_train)

    # loss
    l = loss(y_train, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

   # zero the gradients after updating
    optimizer.zero_grad()
    
#     if epoch % 10 == 0:
print('loss = ', l )    
print()
print("model parameterrs for best accuracy ------------------------")
# l= model.named_parameters()
# print(l)
# for i,j in model.named_parameters():
#             print(i,j )
#print(model.weight,model.bias)

print("hyper parameters- for best accuracy --------------------------------")
print("learning rate",learning_rate,    "number of epochs:",epoch)
output=model(X_test[:1,])
print(output)

y_predicted_cls = output.round()
print("h",y_predicted_cls)
print(y_test[:1])
acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
print(f'accuracy: {acc.item():.4f}')



torch.Size([1600, 784])
befr t tensor([0.0197], grad_fn=<AddBackward0>)
loss =  tensor(0.4076, grad_fn=<MseLossBackward0>)

model parameterrs for best accuracy ------------------------
hyper parameters- for best accuracy --------------------------------
learning rate 0.001 number of epochs: 99
tensor([[0.1404]], grad_fn=<AddmmBackward0>)
h tensor([[0.]], grad_fn=<RoundBackward0>)
tensor([[1.]])
accuracy: 0.5475


In [8]:
from matplotlib import pyplot
import torch
import torch.nn as nn
# X = torch.tensor(, dtype=torch.float32)
# Y = torch.tensor(y, dtype=torch.float32)

print(X.shape)
n_samples, n_features = X.shape
print(n_features)
# 0) create a test sample-----
# print(X)
'''
class Model(nn.Module):
    def __init__(self,):
        
         super().__init__()
         self.hidden = nn.Linear(n_features, 64)
         self.output = nn.Linear(64, 1)
         self.sigmoid = nn.Sigmoid()
         self.softmax = nn.Softmax(dim=0)
    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

model = Model()'''



input_size = 784
hidden_sizes =64
output_size = 10
# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes, output_size),
                      nn.Softmax(dim=1))
# print(model)
# model = nn.Sequential(nn.Linear(n_features,256),nn.Linear(256,1))

# 2) Define loss and optimizer
learning_rate = 0.01
n_iters = 100
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop
for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X)
    #print(y_predicted.shape)
    # loss
    l = loss(y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

   # zero the gradients after updating
    optimizer.zero_grad()
print('loss = ', l )    
print()
print("model parameterrs for best accuracy ------------------------")
l= model.named_parameters()
print(l)
# for i,j in model.named_parameters():
#             print(i,j )
# print(model.weight,model.bias)

print("hyper parameters- for best accuracy --------------------------------")
print("learning rate",learning_rate,    "number of epochs:",epoch)


output=model(X[:1])
print(output)

# y_predicted_cls = output.round()
# acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
# print(f'accuracy: {acc.item():.4f}')


torch.Size([2000, 784])
784


/home/local/ZOHOCORP/abdul-pt6532/miniconda3/envs/py_torch/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2000, 10])) that is different to the input size (torch.Size([2000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss =  tensor(0.4093, grad_fn=<MseLossBackward0>)

model parameterrs for best accuracy ------------------------
<generator object Module.named_parameters at 0x7fc38c8784a0>
hyper parameters- for best accuracy --------------------------------
learning rate 0.01 number of epochs: 99
tensor([[0.1083, 0.0876, 0.0916, 0.1058, 0.1087, 0.1036, 0.0972, 0.0957, 0.0900,
         0.1116]], grad_fn=<SoftmaxBackward0>)


In [7]:
learning_rate/=2
n_iters = 100
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop

for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X)

    # loss
    l = loss(y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

   # zero the gradients after updating
    optimizer.zero_grad()
    
#     if epoch % 10 == 0:
print('loss = ', l )    
print()
print("model parameterrs for best accuracy ------------------------")
# l= model.named_parameters()
# print(l)
# for i,j in model.named_parameters():
#             print(i,j )
print()
print("hyper parameters- for best accuracy --------------------------------")
print("learning rate",learning_rate,"number of epochs:",epoch)
output=model(X)
print(output)



loss =  tensor(0.4092, grad_fn=<MseLossBackward0>)

model parameterrs for best accuracy ------------------------

hyper parameters- for best accuracy --------------------------------
learning rate 0.005 number of epochs: 99
tensor([[0.0941, 0.1137, 0.0929,  ..., 0.1071, 0.0972, 0.1053],
        [0.0927, 0.1109, 0.0913,  ..., 0.1057, 0.1011, 0.1068],
        [0.0944, 0.1121, 0.0915,  ..., 0.1057, 0.1020, 0.1031],
        ...,
        [0.0924, 0.1109, 0.0889,  ..., 0.1081, 0.1018, 0.1055],
        [0.0927, 0.1125, 0.0917,  ..., 0.1058, 0.1013, 0.1010],
        [0.0941, 0.1117, 0.0904,  ..., 0.1055, 0.1015, 0.1034]],
       grad_fn=<SoftmaxBackward0>)
